In [2]:
print("OK!")

OK!


In [30]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [31]:
PINECONE_API_KEY="6f043c11-9675-47ee-b7ab-24161ed91eee"
PINECONE_API_ENV="aws-us-east-1"

In [32]:
#Extract data from the pdf file
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [33]:
extracted_data = load_pdf("data/")

In [7]:
#extracted_data

In [34]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [35]:
text_chunk = text_split(extracted_data)
print("length of the text chunk : " ,len(text_chunk))

length of the text chunk :  7020


In [10]:
#text_chunk

In [36]:
#Download Hugging Face Embeddings
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [37]:
embeddings = download_hugging_face_embedding()

In [38]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [39]:
query_result=embeddings.embed_query("Hello World")
print(len(query_result))
#query_result

384


In [41]:

#Initializing the Pinecone
from pinecone import Pinecone, ServerlessSpec

from langchain.vectorstores import Pinecone as PC

import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medical-chatbot"
index = pc.Index(index_name)
#pinecone_index = PC.from_existing_index(index_name, embeddings, pc)



In [45]:
for i, t in zip(range(len(text_chunk)), text_chunk):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunk[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

In [60]:
# Embed the query text
def embed_query(query, embeddings):
    query_embedding = embeddings.embed_query(query)
    return query_embedding

In [66]:
# Perform a similarity search in the Pinecone index
def similarity_search(query_embedding, pinecone_index, top_k=5):
    results = pinecone_index.similarity_search(query_embedding, top_k)
    return results

In [62]:


# Define your query text
query_text = "What are Allergies?"
query_embedding = embed_query(query_text, embeddings)


In [68]:

# Perform the similarity search
#search_results = similarity_search(query_embedding, pinecone_index, 3)


In [70]:

#If we already have an index we can load it like this
docsearch=PC.from_existing_index(index_name, embeddings)
docsearch

In [ ]:

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)